In [1]:
import os
import tensorflow as tf
from tensorflow.keras import backend
from sagemaker.tensorflow import TensorFlowModel

In [2]:
from sagemaker.session import Session
import sagemaker

In [3]:
role = sagemaker.get_execution_role()

In [4]:
model_data = 's3://yourbucket/model.tar.gz'


In [5]:
model = TensorFlowModel(model_data=model_data, 
                        framework_version='2.8', role=role)

In [6]:
INSTANCE_TYPE = 'ml.m5.xlarge'
ENDPOINT_NAME = 'yolov5l-demo'

predictor = model.deploy(initial_instance_count=1, 
                         instance_type=INSTANCE_TYPE,
                         endpoint_name=ENDPOINT_NAME)

---!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import json
import boto3, botocore
from PIL import Image
import io

In [ ]:
def prep_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height,width = img.shape[0], img.shape[1]

    top_pad = bot_pad = height % 640 // 2
    left_pad = right_pad = width % 640 // 2

    img_padded = cv2.copyMakeBorder(img, top_pad, bot_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=[114,114,114])
    img_padded_and_resized = cv2.resize(img_padded,(640,640))
    #calculate border padding

    img = cv2.resize(img, (4032,3040))

    plt.imshow(img_padded_and_resized)
    return img_padded_and_resized

In [ ]:
img = prep_image('cat_dog.jpg')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.axis('off')

In [ ]:
config = botocore.config.Config(read_timeout=500)
runtime = boto3.client('runtime.sagemaker', config=config)

In [ ]:
data = np.array(img.astype(np.float16)/255.)
payload = json.dumps([data.tolist()])

response = runtime.invoke_endpoint(EndpointName='yolov5l-demo', ContentType='application/json', Body=payload)

In [ ]:
result = json.loads(response['Body'].read().decode())

In [ ]:
indices = np.where(np.array(result['predictions'][0]['output_1']) > 0.5)
xywh = np.array(result['predictions'][0]['output_0'])[indices]
xywh[:,0] *= 640
xywh[:,1] *= 640
xywh[:,2] *= 640
xywh[:,3] *= 640
xywh = xywh.astype(int)

scores = np.array(result['predictions'][0]['output_1'])[indices]
classes = np.array(result['predictions'][0]['output_2'])[indices]

In [ ]:
class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush'] 

In [ ]:
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONTSCALE = .6
WHITE = (255, 255, 255)
THICKNESS = 2
for idx, rect in enumerate(xywh):
    img = cv2.rectangle(img,
              (rect[0], rect[1]-5),
              (rect[2], rect[3]), thickness=2, color = (255,0,0))
    
    class_idx = int(classes[idx])
    img = cv2.putText(img, 
                      f'{class_names[class_idx]}: {scores[idx]:0.3f}',
                      (rect[0],rect[1]),
                      FONT,
                      FONTSCALE,
                      WHITE,
                      THICKNESS)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.axis('off')

In [ ]:
import boto3, botocore
s3_client = boto3.client(
    's3',
    aws_access_key_id='YOUR ACCESS KEY',
    aws_secret_access_key='YOUR SECRET KEY'
)

In [ ]:
encode_img = cv2.imwrite('saved_image.jpg', img)
colored_img = Image.fromarray(img).convert('RGB')
out_img = io.BytesIO()
colored_img.save(out_img, format='jpeg')
out_img.seek(0)  

In [ ]:
try:
    response = s3_client.put_object(Bucket='yourbucket', Key='saved_image.jpg', Body=out_img)
    print("File saved to S3:", response)
except Exception as e:
    print("Error:", e)

In [ ]:
classes = classes.astype(int)

In [ ]:
for item in classes:
    print(class_names[item])